# Setup spark

In [90]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] ="--conf spark.driver.memory=2g  pyspark-shell"

from pyspark.sql import SparkSession
from pyspark.sql.functions import flatten
from pyspark.sql.types import (StructType, StructField, StringType, 
                                FloatType, DateType, IntegerType, ArrayType)
spark = SparkSession \
        .builder \
        .master("local[*]") \
        .appName("BDA assignment") \
        .getOrCreate()

# Imports

In [73]:
from typing import NamedTuple, Final, List
#from lxml import etree
import xml.etree.ElementTree as ET
from itertools import islice, chain, combinations
import argparse
import traceback
import bleach
import html
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.util import pr
import string
import random
import hashlib
import pandas as pd
import numpy as np
import hashlib
import json
import sys

# Constants

In [3]:
SHINGLE_SIZE: Final = 5
SAMPLES: Final = 1000

In [4]:
class comment_tuple(NamedTuple):
    id: int
    #owner_id: int
    post_type: int
    score: int
    text: str

class shingle_set(NamedTuple):
    id: int
    shingles: frozenset[tuple]

class similarity(NamedTuple):
    id_set1: int
    id_set2: int
    similarity: float

# Read and clean XML

In [5]:
def set_schema():
    """
    Define the schema for the DataFrame
    """
    schema_list = []
    schema_list.append(StructField("Id", IntegerType(), True))
    #schema_list.append(StructField("PostTypeId", IntegerType(), True))
    #schema_list.append(StructField("Score", IntegerType(), True))
    schema_list.append(StructField("Body", StringType(), True))
    
    return StructType(schema_list)

def parse_post(rdd):
    results = []
    root = ET.fromstring(rdd[0])

    for elem in root.findall('row'):
        rec = []
        #print("Found row")
        assert elem.text is None, "The row wasn't empty"
        rec.append(int(elem.attrib["Id"]))
        #int(elem.attrib["OwnerUserId"]),
        #rec.append(int(elem.attrib["PostTypeId"])),
        #rec.append(int(elem.attrib["Score"])),
        rec.append(bleach.clean(elem.attrib["Body"], strip=True))
        #rec.append(elem.attrib["Body"])

        #elem.clear()
        #while elem.getprevious() is not None:
        #    del elem.getparent()[0]
        results.append(rec)

    return results

In [6]:
filename = "cstheory_posts.bakxml"
chunksize = 1024

file_rdd = spark.read.text(filename, wholetext=True).rdd
dataset = file_rdd.flatMap(parse_post)

In [7]:
dataset.persist()
schema = set_schema()
df_ds = dataset.toDF(schema)
df_ds.show()

#df_posts = records_rdd.toDF(schema)
#coll = records_rdd.collect()

+---+--------------------+
| Id|                Body|
+---+--------------------+
|  2|I have a dataset ...|
|  3|A particular prog...|
|  4|What is the follo...|
|  5|Can the divide an...|
|  6|Is anyone aware o...|
|  7|In general, the q...|
|  8|If I understand t...|
|  9|<a href="http://w...|
| 10|There was recentl...|
| 11|Functional progra...|
| 12|I took a class on...|
| 13|It's possible tha...|
| 14|Other than going ...|
| 15|[This question ha...|
| 16|See the <a href="...|
| 17|it is often said ...|
| 18|In one word: No.
...|
| 19|In short, I would...|
| 20|It's easy to prov...|
| 21|Any &quot;cutting...|
+---+--------------------+
only showing top 20 rows



# Shingling

In [11]:
class shingler:
    """
    Class that contain a tokenizer and stopwords to make shingling easier.
    """
    def __init__(self):
        self.stopwords = set(stopwords.words('english'))

    def __enter__(self):
        return self

    def __exit__(self, exc_type, exc_value, traceback):
        return True

    def tokenize(self, text: str) -> List[str]:
        text = text.translate(str.maketrans('', '', string.punctuation))
        # text_nop = text.split()
        text_nop = word_tokenize(text)
        filtered_words = []

        for word in text_nop:
            if word not in self.stopwords:
                filtered_words.append(word.lower())
        
        return filtered_words

    #def create_shingle(self, input_comment: comment_tuple, shingle_size: int) -> frozenset[tuple]:
    #    tokens = self.tokenize(input_comment.text)
    #    comment_length = len(tokens)
    #    shingles =  frozenset(tuple(tokens[i:(i + shingle_size)]) for i in range(comment_length - shingle_size + 1))
    
    def create_shingle(self, post: str, shingle_size: int) -> list[list]:
        tokens = self.tokenize(post)
        comment_length = len(tokens)
        shingle_set = frozenset(tuple(tokens[i:(i + shingle_size)]) for i in range(comment_length - shingle_size + 1))
        return list(shingle_set)
        

def shingle_map(row):
    ds_shingler = shingler()
    return (row[0], ds_shingler.create_shingle(row[1], SHINGLE_SIZE)
)

def set_shingle_schema():
    """
    Define the schema for the DataFrame
    """
    schema_list = []
    schema_list.append(StructField("Id", IntegerType(), True))
    #schema_list.append(StructField("PostTypeId", IntegerType(), True))
    #schema_list.append(StructField("Score", IntegerType(), True))
    schema_list.append(StructField("Shingles", ArrayType(ArrayType(StringType()), True)))
    return StructType(schema_list)

In [109]:
schema = set_shingle_schema()
shingle_rdd = dataset.map(shingle_map)
df_shingle = shingle_rdd.toDF(schema)
#df_shingle.show(truncate=False)

# MinHashing

In [ ]:
# Get doc list
doc_list = np.array(flatten(df_shingle.select('Shingles').collect()))
print(doc_list[:10])

In [132]:
# Transform posts to characteristic matrix
# Make feature set matrix
# Minhash
# Make Minhash Matrix
# LSH
import random
SIGNATURE_SIZE: Final = 50
HASH_PRIME: Final = (1 << 61) - 1
MAX_HASH: Final = sys.maxsize
HASH_RANGE: Final = sys.maxsize
SEED: Final = 193120



generator = np.random.RandomState(SEED)
salts = [generator.randint(1, HASH_PRIME) for _ in range(SIGNATURE_SIZE)]
permutations = [generator.randint(1, HASH_PRIME) for _ in range(SIGNATURE_SIZE)]

#permutations = [[generator.randint(1, HASH_PRIME) for _ in range(SIGNATURE_SIZE)],
#                [generator.randint(0, HASH_PRIME) for _ in range(SIGNATURE_SIZE)]]

print(MAX_HASH)
print(salts)
print(permutations)

def hash_func(data, salt):
    return int.from_bytes(hashlib.md5(int.to_bytes(salt, 8, byteorder="big")  + json.dumps(data).encode()).digest()[:8], byteorder="big")
    
def min_hasher(row):
    sig = np.full((SIGNATURE_SIZE), MAX_HASH)
    for shingle in row[1]:
        for i in range(SIGNATURE_SIZE):
            hash_val = hash_func(tuple(shingle), salts[i])
            sig[i] = min(hash_val, sig[i])
    return (row[0], sig)

9223372036854775807
[2096149490282617182, 2269561747909081125, 607930631362827344, 2092219096079086764, 1809674349804163182, 1338257575307146427, 1928960101259846486, 315389730586719761, 559989775987727879, 1480681506970345963, 2261419810219035797, 2243558520118467258, 1931388090443637456, 2212609764843106243, 350380123870349570, 1304243101527343489, 1592214487878087633, 1277783616767142909, 2280831058830631016, 1360369692187081524, 494790709208836929, 610180859697606999, 2189755580139374733, 480413561091986636, 2161153900773804701, 1002261504377444921, 696261638825223443, 2101683783981916985, 1599279409116898063, 2253616327139758357, 112451947857840231, 102391885042708822, 2213763498531204161, 951137115894174460, 733117990283907917, 106046510276105074, 2004829538990433145, 1098029479313378209, 276824724253688489, 211516063877708228, 2233803146264722951, 2025602953273918832, 142435781221445861, 1284554568422474643, 752807239468083502, 311725101681684327, 171144034284598831, 26890183545

In [133]:
hash_rdd = shingle_rdd.map(min_hasher)

for elem in hash_rdd.take(3):
    print(elem)

(2, array([ 189702638683285394,  143889349851350696,  299970645845496758,
       1012789781694830188,  139233372471519040,  128758974008313556,
        155065139646647311,   50898033533516211,  108221274923554810,
        268327233853184876,    4830045840359835,  753944583133075645,
         38613772992196326,  390893800529103552,   60125358139790662,
        471037100977378370,  446785690323532210,  246669865872026019,
        664716957311820284,  387620932667581972,  703407128874244779,
        386799754678351464,   92916554796120362,   70671722885316164,
         20867341287560508,  256974338996562518,  209282355311288669,
         34218393967450437, 1248669209218069845,  265782525961938765,
        377931790261232701,  285697169564483964,  200994712323080486,
        528388419207376598,  431910206058034197,  100508840466194519,
        210482763853889684,  908617023945961952,  662583839978245135,
         86390792287761791,   54223620111278424,   24081312883022476,
        13154916

# Exit Spark

In [ ]:
#spark.stop()